# Subset CMIP6 Datasets with xarray

xarray: http://xarray.pydata.org/en/stable/index.html

## Example Notebooks

* xclim: https://nbviewer.jupyter.org/github/Ouranosinc/xclim/tree/master/docs/notebooks/

## Search CMIP6 Dataset

using: https://esgf-pyclient.readthedocs.io/en/latest/index.html

In [ ]:
from pyesgf.search import SearchConnection
conn = SearchConnection('https://esgf-data.dkrz.de/esg-search', distrib=True)

In [ ]:
ctx = conn.new_context(
    project='CMIP6', 
    source_id='UKESM1-0-LL', 
    experiment_id='historical', 
    variable='tas', 
    frequency='mon', 
    variant_label='r1i1p1f2')
ctx.hit_count

In [ ]:
result = ctx.search()[1]
result.dataset_id

In [ ]:
files = result.file_context().search()
for file in files:
    print(file.opendap_url)

## Subset single dataset with xarray

Using OpenDAP: http://xarray.pydata.org/en/stable/io.html?highlight=opendap#opendap

In [ ]:
import xarray as xr
ds = xr.open_dataset(files[0].opendap_url, chunks={'time': 1200})
print(ds)

In [ ]:
da = ds['tas']
da = da.isel(time=slice(0, 1))
da = da.sel(lat=slice(-50, 50), lon=slice(0, 50))


In [ ]:
%matplotlib inline
da.plot()

## Subset over multiple datasets


In [ ]:
ds_agg = xr.open_mfdataset([files[0].opendap_url, files[1].opendap_url], chunks={'time': 1980})
print(ds_agg)

In [ ]:
da = ds_agg['tas']
da = da.isel(time=slice(1200, 1201))
da = da.sel(lat=slice(-50, 50), lon=slice(0, 50))

In [ ]:
da.plot()

## Regrid with xESMF

xESMF based on xarray: https://xesmf.readthedocs.io/en/latest/Rectilinear_grid.html

### Input Grid

Grid resolution: 1.25 x 1.875

In [ ]:
ds['lat'].values, ds['lon'].values

### Output Grid

resample to 2.5x3.75

In [ ]:
import numpy as np

ds_out = xr.Dataset({'lat': (['lat'], np.arange(-89.375, 89.375, 2.5)),
                     'lon': (['lon'], np.arange(0.9375, 359.0625, 3.75)),
                    }
                   )
ds_out

### Perform Regridder

In [ ]:
import xesmf as xe

regridder = xe.Regridder(ds, ds_out, 'bilinear')
regridder

In [ ]:
da_out = regridder(ds.tas)
da_out

In [ ]:
da_out.isel(time=0).plot()